In [1]:
import pyspark

In [2]:
import pandas as pd

In [3]:
import pyspark
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pyspark.sql.functions as sf
appName= "hive_pyspark"
master= "local"

## create sparksession

In [4]:
from pyspark.sql import SparkSession

In [59]:
#spark=SparkSession.builder.appName('Dataframe').getOrCreate()

In [60]:
#spark

In [5]:
spark = SparkSession.builder.master(master).appName(appName).enableHiveSupport().getOrCreate()

22/04/04 12:04:23 WARN Utils: Your hostname, easebuzz-Inspiron-3501 resolves to a loopback address: 127.0.1.1; using 10.10.2.6 instead (on interface wlp0s20f3)
22/04/04 12:04:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/04 12:04:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
df=spark.sql("show databases")
df.show()

22/04/04 12:04:31 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
22/04/04 12:04:31 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
22/04/04 12:04:34 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
22/04/04 12:04:34 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore easebuzz@127.0.1.1


+---------+
|namespace|
+---------+
|  default|
+---------+



In [63]:
datafile=spark.read.csv("/media/easebuzz/DATA/Code/raw_files/Rbl_processed.csv",header=True)
datafile.show(5)
#datafile.write.saveAsTable("test_table")

+----------------+----------+-----------------------+------------+---------+----------+--------------+---------+--------------------+--------------------+
|Transaction Date|Value Date|Transaction Discription|Referance No|Debit Amt|Credit Amt|   Running Bal|Bank-name|           file-name|       date_ingested|
+----------------+----------+-----------------------+------------+---------+----------+--------------+---------+--------------------+--------------------+
|01-12-2021 06:10|01-12-2021|   NEFT/000214522423...|        null|     3.72|      null|2,47,97,607.12| RBL-Bank|s3://reconcilatio...|2022-03-11 19:17:...|
|01-12-2021 06:10|01-12-2021|   NEFT/000214522442...|        null|     7.46|      null|2,47,97,599.66| RBL-Bank|s3://reconcilatio...|2022-03-11 19:17:...|
|01-12-2021 06:10|01-12-2021|   NEFT/000214522424...|        null|     11.2|      null|2,47,97,588.46| RBL-Bank|s3://reconcilatio...|2022-03-11 19:17:...|
|01-12-2021 06:10|01-12-2021|   NEFT/000214522466...|        null| 3,0

In [64]:
#converting dataframe perticular column to a list
list1=([data[0] for data in datafile.select('Transaction Discription').collect()])

In [65]:

df2 = datafile.where(
    datafile['Transaction Discription'].rlike("|".join(["(" + pat + ")" for pat in list1]))
)

In [66]:
df2.show()

+----------------+----------+-----------------------+------------+-----------+----------+--------------+---------+--------------------+--------------------+
|Transaction Date|Value Date|Transaction Discription|Referance No|  Debit Amt|Credit Amt|   Running Bal|Bank-name|           file-name|       date_ingested|
+----------------+----------+-----------------------+------------+-----------+----------+--------------+---------+--------------------+--------------------+
|01-12-2021 06:10|01-12-2021|   NEFT/000214522423...|        null|       3.72|      null|2,47,97,607.12| RBL-Bank|s3://reconcilatio...|2022-03-11 19:17:...|
|01-12-2021 06:10|01-12-2021|   NEFT/000214522442...|        null|       7.46|      null|2,47,97,599.66| RBL-Bank|s3://reconcilatio...|2022-03-11 19:17:...|
|01-12-2021 06:10|01-12-2021|   NEFT/000214522424...|        null|       11.2|      null|2,47,97,588.46| RBL-Bank|s3://reconcilatio...|2022-03-11 19:17:...|
|01-12-2021 06:10|01-12-2021|   NEFT/000214522466...|     

In [67]:
df2.dtypes

[('Transaction Date', 'string'),
 ('Value Date', 'string'),
 ('Transaction Discription', 'string'),
 ('Referance No', 'string'),
 ('Debit Amt', 'string'),
 ('Credit Amt', 'string'),
 ('Running Bal', 'string'),
 ('Bank-name', 'string'),
 ('file-name', 'string'),
 ('date_ingested', 'string')]

# --------------------------------------------------------------------------------------------------------------------------------

In [68]:
dfregset= spark.read.csv("/media/easebuzz/DATA/Code/raw_files/Source_regiment.csv",header=True)
dfregset.show()

+--------------------+--------+
|           Regiments|    Arms|
+--------------------+--------+
|      Assam regiment|infantry|
|     Garehwal rifles|infantry|
|   Rajputana rifles |infantry|
|     Rajput regiment|infantry|
|Maratha light inf...|infantry|
| Sikh light infantry|infantry|
|       Sikh regiment|infantry|
|       Para regiment|infantry|
| para special forces|infantry|
|     Signal regiment|    ENTC|
|       508 regiment |Mech-inf|
|     Madras Regiment|infantry|
|          Granediers|infantry|
+--------------------+--------+



In [69]:
dffilterset= spark.read.csv("/media/easebuzz/DATA/Code/raw_files/filter_reg.csv",header=True)
dffilterset.show()

+----------+
|filter_reg|
+----------+
|     Assam|
|  Garehwal|
|       508|
+----------+



In [70]:
fillist=([data[0] for data in dffilterset.select('filter_reg').collect()])
print(fillist)


['Assam', 'Garehwal', '508']


In [71]:
afterfil = dfregset.where(
    dfregset['Regiments'].rlike("|".join(["(" + pat + ")" for pat in fillist]))
)
afterfil.show()


+---------------+--------+
|      Regiments|    Arms|
+---------------+--------+
| Assam regiment|infantry|
|Garehwal rifles|infantry|
|  508 regiment |Mech-inf|
+---------------+--------+



In [72]:
dfregset.subtract(afterfil).show()

+--------------------+--------+
|           Regiments|    Arms|
+--------------------+--------+
|     Madras Regiment|infantry|
|     Signal regiment|    ENTC|
|          Granediers|infantry|
|   Rajputana rifles |infantry|
|       Sikh regiment|infantry|
|     Rajput regiment|infantry|
| Sikh light infantry|infantry|
| para special forces|infantry|
|Maratha light inf...|infantry|
|       Para regiment|infantry|
+--------------------+--------+



In [73]:
#dftest = dfregset.merge(afterfil, how = 'outer' ,indicator=True).loc[lambda x : x['merge']=='left_only']
#dftest = df1.merge(df2, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']

In [74]:
#diff_df = pd.merge(dfregset, afterfil, how='outer', indicator='Exist')

#diff_df = diff_df.loc[diff_df['Exist'] != 'both']

#df_diff = pd.concat([dfregset],afterfil).drop_duplicates(keep=False)

In [75]:
#dfregset=pd.concat[afterfil]

In [76]:
receipt = pd.read_excel("/media/easebuzz/DATA/Code/raw_files/receipt-14-03-2022-yespart A FINAL.xls")
print(receipt.head)

<bound method NDFrame.head of                       Date                                        Description  \
0               14/03/2022                                              Bin-1   
1               14/03/2022                                              Bin-2   
2               14/03/2022                                              Bin-3   
3               14/03/2022                     UPI Transaction dtd 12/03/2022   
4               14/03/2022                     UPI Transaction dtd 11/03/2022   
5               14/03/2022                                             Yes NB   
6               14/03/2022                                              Yes-1   
7               14/03/2022                                              Yes-2   
8               14/03/2022                                              Yes-3   
9               14/03/2022                                              Yes-4   
10              14/03/2022                                              Yes-5  

# ------------------------------------------------------------------------------------------------------------------------

In [77]:
Yesstatements =spark.read.csv('/media/easebuzz/DATA/Code/Processed_files/bankstatements_test.csv',header=True)
Yesstatements.show()

+-------------------+--------------------+--------------------+
|   transaction_date|          value_date|            NOTATION|
+-------------------+--------------------+--------------------+
|2021-11-01 00:25:00|2021-10-30 18:30:...|R/UPI/13051309360...|
|2021-11-01 05:57:00|2021-10-31 18:30:...|R/UPI/13051309735...|
|2021-11-01 09:52:00|2021-10-31 18:30:...|RTGS/UTIBH2130521...|
|2021-11-01 12:29:00|2021-10-31 18:30:...|IMPS 130512021968...|
|2021-11-01 06:25:00|2021-10-31 18:30:...|R/UPI/13051309792...|
|2021-11-01 13:06:00|2021-10-31 18:30:...|UPI/130555786563/...|
|2021-11-01 15:16:00|2021-10-31 18:30:...|IMPS 130515581433...|
|2021-11-01 17:18:00|2021-10-31 18:30:...|    TR9E6B0CDD65724F|
|2021-11-01 18:08:00|2021-10-31 18:30:...|IMPS 130518852314...|
|2021-11-01 20:14:00|2021-10-31 18:30:...|NEFT/INDBN0111485...|
|2021-11-01 23:17:00|2021-10-31 18:30:...|IMPS 130523849594...|
|2021-11-02 11:31:00|2021-11-01 18:30:...|R-130611071368-RE...|
|2021-11-02 12:38:00|2021-11-01 18:30:..

In [78]:
Utr =spark.read.csv("/media/easebuzz/DATA/Code/Processed_files/UtR_test.csv",header=True)
Utr.show(5)

+----------------+
|             UTR|
+----------------+
|    130513093601|
|    130513097355|
|UTIBH21305219225|
|    130512021968|
|    130513097925|
+----------------+
only showing top 5 rows



In [79]:
Utrlist=([data[0] for data in Utr.select('UTR').collect()])
print(Utrlist)


['130513093601', '130513097355', 'UTIBH21305219225', '130512021968', '130513097925', '130555786563', '130515581433', '130518852314', 'INDBN01114855660', '130523849594', '130611071368', '130613047373', 'SBIN321305229549', 'IBKLH21305600499', 'ICICH21306149418', '130619650378', 'HDFCH21306191224', '130622355229', '130513093616', '130513094584', '130513096509', '130509029235', '130509372224', '130710474439', 'N307211701074022', '130716067880', '130719049699', '130719050613', 'UTIBH21307245061', '130718510506', 'R-130718043751-REVERSAL-DP', 'IMPS 130719939340 FROM JUSTDIAL']


In [80]:
filtered_list=[]
for ele in Utrlist:
    if ele != None:
        filtered_list.append(ele)
print(filtered_list)

['130513093601', '130513097355', 'UTIBH21305219225', '130512021968', '130513097925', '130555786563', '130515581433', '130518852314', 'INDBN01114855660', '130523849594', '130611071368', '130613047373', 'SBIN321305229549', 'IBKLH21305600499', 'ICICH21306149418', '130619650378', 'HDFCH21306191224', '130622355229', '130513093616', '130513094584', '130513096509', '130509029235', '130509372224', '130710474439', 'N307211701074022', '130716067880', '130719049699', '130719050613', 'UTIBH21307245061', '130718510506', 'R-130718043751-REVERSAL-DP', 'IMPS 130719939340 FROM JUSTDIAL']


In [81]:
afterfil = Yesstatements.where(
    Yesstatements['NOTATION'].rlike("|".join(["(" + pat + ")" for pat in filtered_list]))
)

In [82]:
unmatched=Yesstatements.subtract(afterfil)

In [83]:
afterfil= afterfil.withColumn('Status', sf.lit('Matched'))

In [84]:
unmatched= unmatched.withColumn('Status', sf.lit('Unmatched'))

In [85]:
unmatched.show()

+-------------------+--------------------+--------------------+---------+
|   transaction_date|          value_date|            NOTATION|   Status|
+-------------------+--------------------+--------------------+---------+
|2021-11-09 17:32:00|2021-11-08 18:30:...|RTGS/IDFBH2131355...|Unmatched|
|2021-11-19 15:53:00|2021-11-18 18:30:...|UPI/132377504949/...|Unmatched|
|2021-11-05 20:27:00|2021-11-04 18:30:...|IMPS 130920591776...|Unmatched|
|2021-11-13 19:11:00|2021-11-12 18:30:...|IMPS 131719977880...|Unmatched|
|2021-11-01 19:25:00|2021-10-31 18:30:...|IMPS 130519185896...|Unmatched|
|2021-11-06 18:36:00|2021-11-05 18:30:...|RTGS/BCBMH2131073...|Unmatched|
|2021-11-03 19:40:00|2021-11-02 18:30:...|R-130719068598-RE...|Unmatched|
|2021-11-11 20:34:00|2021-11-10 18:30:...|RTGS/IDFBH2131556...|Unmatched|
|2021-11-02 13:23:00|2021-11-01 18:30:...|IMPS 130613116845...|Unmatched|
|2021-11-15 12:20:00|2021-11-14 18:30:...|IMPS 131912819043...|Unmatched|
|2021-11-12 10:47:00|2021-11-11 18:30:

In [86]:
afterfil.show()

+-------------------+--------------------+--------------------+-------+
|   transaction_date|          value_date|            NOTATION| Status|
+-------------------+--------------------+--------------------+-------+
|2021-11-01 00:25:00|2021-10-30 18:30:...|R/UPI/13051309360...|Matched|
|2021-11-01 05:57:00|2021-10-31 18:30:...|R/UPI/13051309735...|Matched|
|2021-11-01 09:52:00|2021-10-31 18:30:...|RTGS/UTIBH2130521...|Matched|
|2021-11-01 12:29:00|2021-10-31 18:30:...|IMPS 130512021968...|Matched|
|2021-11-01 06:25:00|2021-10-31 18:30:...|R/UPI/13051309792...|Matched|
|2021-11-01 13:06:00|2021-10-31 18:30:...|UPI/130555786563/...|Matched|
|2021-11-01 15:16:00|2021-10-31 18:30:...|IMPS 130515581433...|Matched|
|2021-11-01 18:08:00|2021-10-31 18:30:...|IMPS 130518852314...|Matched|
|2021-11-01 20:14:00|2021-10-31 18:30:...|NEFT/INDBN0111485...|Matched|
|2021-11-01 23:17:00|2021-10-31 18:30:...|IMPS 130523849594...|Matched|
|2021-11-02 11:31:00|2021-11-01 18:30:...|R-130611071368-RE...|M

In [87]:
print((afterfil.count(), len(afterfil.columns)))

(32, 4)


In [88]:
print((unmatched.count(), len(unmatched.columns)))

(168, 4)


In [89]:
print((Yesstatements.count(), len(Yesstatements.columns)))

(200, 3)


# ---------------------------------------------------------------------------------------------------------------------------------

In [90]:
#Yesstatements.withColumn("MATCHING",col("NOTATION").rlike(filtered_list.mkString("|"))).show(truncate = false)

In [91]:
#Yesstatements.withColumn("contains_cat_or_dog",col("phrase").rlike(animals.mkString("|"))).show(truncate = false)

# ------------------------------------------------------------------------------------------------------------------------------

In [92]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

In [93]:
data2 = [("James","","Smith","36636","M",3000,3000),
    ("Michael","Rose","","40288","M",4000,4500),
    ("Robert","","Williams","42114","M",4000,4000),
    ("Maria","Anne","Jones","39192","F",4000,3500),
    ("Jen","Mary","Brown","","F",-1,4000)
  ]

In [94]:
schema = StructType([ \
    StructField("firstname",StringType(),True), \
    StructField("middlename",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("id", StringType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", StringType(), True), \
    StructField("refnum",StringType(), True) \
  ])

In [95]:
testdf=spark.createDataFrame(data2,schema=schema)

In [96]:
testdf.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- refnum: string (nullable = true)



In [97]:
#testdf.show(truncate=False)

In [98]:
receipt2 = pd.read_excel("/media/easebuzz/DATA/Code/raw_files/YESUPI_RECEIPT-Jan-2022.xls")
print(receipt.head)

<bound method NDFrame.head of                       Date                                        Description  \
0               14/03/2022                                              Bin-1   
1               14/03/2022                                              Bin-2   
2               14/03/2022                                              Bin-3   
3               14/03/2022                     UPI Transaction dtd 12/03/2022   
4               14/03/2022                     UPI Transaction dtd 11/03/2022   
5               14/03/2022                                             Yes NB   
6               14/03/2022                                              Yes-1   
7               14/03/2022                                              Yes-2   
8               14/03/2022                                              Yes-3   
9               14/03/2022                                              Yes-4   
10              14/03/2022                                              Yes-5  

In [99]:
statements = spark.read.csv("/media/easebuzz/DATA/Code/raw_files/YESNSTAT_01Jan 22_to_10Jan22.csv",header=True)
datafile.show(5)

AnalysisException: Path does not exist: file:/media/easebuzz/DATA/Code/raw_files/YESNSTAT_01Jan 22_to_10Jan22.csv

In [ ]:
print((statements.count(), len(statements.columns)))

(707824, 1)


In [ ]:
ledger = spark.read.csv("/media/easebuzz/DATA/Code/raw_files/ledger_utr_data.csv",header=True)
ledger.show(5)

+----------------------------+------------------+----------------+
|unique_transaction_reference|transaction_amount|transaction_date|
+----------------------------+------------------+----------------+
|                202929339389|                 1|29-01-2022 15:56|
|                202420152951|                 1|         55:36.0|
|                202214786319|                48|         38:07.1|
|                202101152051|                50|         12:29.0|
|                201117820225|               200|         35:04.6|
+----------------------------+------------------+----------------+
only showing top 5 rows



In [ ]:
print((ledger.count(), len(ledger.columns)))

(1048575, 3)


# --------------------------------------------------------------------------------------------------------------

In [ ]:
mark1 = spark.read.csv("/media/easebuzz/DATA/Code/raw_files/match1.csv",header=True)
mark1= mark1.select('Referance no')
mark2 = spark.read.csv("/media/easebuzz/DATA/Code/raw_files/match2.csv",header=True)
mark2= mark2.select('Referance no')
mark3 = spark.read.csv("/media/easebuzz/DATA/Code/raw_files/Match3.csv",header=True)
mark3= mark3.select('Referance no')
mark4 = spark.read.csv("/media/easebuzz/DATA/Code/raw_files/Match4.csv",header=True)
mark4= mark4.select('Referance no')


In [ ]:
disDF = mark1.union(mark2).distinct()
disDF = disDF.union(mark3).distinct()
disDF = disDF.union(mark4).distinct()

In [ ]:
disDF.printSchema()

root
 |-- Referance no: string (nullable = true)



In [ ]:
print((disDF.count(), len(disDF.columns)))

(1970, 1)


In [ ]:
disDF.toPandas().to_csv('mymatchedcsv.csv')

In [ ]:
mpr_file= spark.read.csv("/media/easebuzz/DATA/Code/raw_files/YESUPI_RECEIPT-Jan-2022.csv",header=True)

In [ ]:
mpr_file.printSchema()

root
 |-- Transaction Date: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- C.D.Flag: string (nullable = true)
 |-- Ref .No: string (nullable = true)
 |-- Wire: string (nullable = true)



In [ ]:
tempList = [] 
for col in mpr_file.columns:
    new_name = col.strip()
    new_name = "".join(new_name.split())
    new_name = new_name.replace('.','') # EDIT
    tempList.append(new_name) #Edit02
print(tempList)
mpr_file = mpr_file.toDF(*tempList)

['TransactionDate', 'Description', 'Amount', 'CDFlag', 'RefNo', 'Wire']


In [ ]:
selectedrows_mpr = mpr_file.select('RefNo')

In [ ]:
matched_files= spark.read.csv("/media/easebuzz/DATA/Code/raw_files/mymatchedcsv.csv",header=True)

In [ ]:
matched_files.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Referance no: string (nullable = true)



In [ ]:
matched_files=matched_files.drop("_c0")

In [ ]:
matched_files.printSchema()

root
 |-- Referance no: string (nullable = true)



In [ ]:
matched_files=matched_files.select('Referance no')

In [ ]:
#selectedrows_matched=matched_files.select('Transaction Descriprtion')

In [ ]:
result=selectedrows_mpr.subtract(matched_files)

In [ ]:
print((result.count(), len(result.columns)))

(3147, 1)


In [ ]:
result.toPandas().to_csv('unmatchedcsv.csv')

# ------------------------------------------------------------------------------------------------------------------------------

In [ ]:
unmatched_csv= spark.read.csv("/media/easebuzz/DATA/Code/raw_files/unmatched_pg.csv",header=True)

In [ ]:
unmatched_csv.printSchema()

root
 |-- Transaction Date: string (nullable = true)
 |-- Value Date: string (nullable = true)
 |-- Transaction Description: string (nullable = true)
 |-- Referance no3: string (nullable = true)
 |-- Debit Amount: string (nullable = true)
 |-- Credit Amount: string (nullable = true)
 |-- Closing Balance6: string (nullable = true)
 |-- Remark: string (nullable = true)
 |-- Transaction Descriprtion: string (nullable = true)
 |-- Closing balance9: string (nullable = true)
 |-- Referance No10: string (nullable = true)
 |-- Status: string (nullable = true)



In [ ]:
#df=df_states.withColumn('joined_column', sf.concat(sf.col('state_name'),sf.lit(' '),sf.col('state_code')))
#df.show()

In [ ]:
#unmatched_csv= unmatched_csv.withColumn('Transaction Date', sf.concat(sf.col('Transaction Date'),sf.lit(' '),sf.col('Transaction Date')))

In [ ]:
#unmatched_csv.toPandas().to_csv('unmatchedcsv.csv')

In [ ]:
from pyspark.sql.functions import concat, lit, col

In [ ]:
#df1.toPandas().to_csv('unmatchedcsv.csv')

In [ ]:
df1 = unmatched_csv.withColumnRenamed("Transaction Descriprtion","Hello")


In [ ]:
df1.printSchema()

root
 |-- Transaction Date: string (nullable = true)
 |-- Value Date: string (nullable = true)
 |-- Transaction Description: string (nullable = true)
 |-- Referance no3: string (nullable = true)
 |-- Debit Amount: string (nullable = true)
 |-- Credit Amount: string (nullable = true)
 |-- Closing Balance6: string (nullable = true)
 |-- Remark: string (nullable = true)
 |-- Hello: string (nullable = true)
 |-- Closing balance9: string (nullable = true)
 |-- Referance No10: string (nullable = true)
 |-- Status: string (nullable = true)



In [ ]:
#df1 = unmatched_csv.withColumnRenamed("Transaction Descriprtion","hello1")

#df1.select(concat(col("hello1"), lit(" "), col("Hello")))

In [ ]:
newDf =
  df1.withColumn(
    "NEW_COLUMN",
    concat(
      when(col("hello1").isNotNull, col("hello1")).otherwise(lit("null")),
      when(col("Hello").isNotNull, col("Hello")).otherwise(lit("null"))))

SyntaxError: invalid syntax (2105765560.py, line 1)

In [ ]:
#df1=df1.select("*", concat(col("Transaction Descriprtion"),col("Hello")).alias("Final"))
#df1.show()

# --------------------------------------------------------------------------------------------------------------------------------

In [ ]:
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql.functions import split

In [ ]:
bstatements = spark.read.csv("/media/easebuzz/DATA/Code/raw_files/Jan bank_stmt_final.csv",header=True)

In [ ]:
print((bstatements.count(), len(bstatements.columns)))

(384, 14)


In [ ]:
Neft_return = bstatements.filter(bstatements.transaction_description.startswith('NEFT-RETURN'))

In [ ]:
print((Neft_return.count(), len(Neft_return.columns)))

(25, 14)


In [ ]:
Neft_return = Neft_return.withColumn("UTR", split(col("transaction_description"), "-").getItem(2))

In [ ]:
Neft_return.show()

+----------------+--------------------+-----------------------+------------+-------------+---------------+-----------------+-------------------+-------+------+-------------+----------------+----+----+----------------+
|transaction_date|          value_date|transaction_description|debit_amount|credit_amount|running_balance|pg_split_verified|wire_split_verified|product|reason|verified_flag|      split_part|_c12|_c13|             UTR|
+----------------+--------------------+-----------------------+------------+-------------+---------------+-----------------+-------------------+-------+------+-------------+----------------+----+----+----------------+
|         32:00.0|2022-01-07 00:00:...|   NEFT-RETURN-N0072...|        null|       198.87|    925681327.5|             null|               null|     PG|  null|            2|N007220948632326|null|null|N007220948632326|
|         32:00.0|2022-01-07 00:00:...|   NEFT-RETURN-N0072...|        null|        97.01|      919515521|             null|    

22/03/29 17:28:47 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: transaction_date, value_date, transaction_description, debit_amount, credit_amount, running_balance, pg_split_verified, wire_split_verified, product, reason, verified_flag, split_part, , 
 Schema: transaction_date, value_date, transaction_description, debit_amount, credit_amount, running_balance, pg_split_verified, wire_split_verified, product, reason, verified_flag, split_part, _c12, _c13
Expected: _c12 but found: 
CSV file: file:///media/easebuzz/DATA/Code/raw_files/Jan%20bank_stmt_final.csv


In [ ]:
TR = bstatements.filter(bstatements.transaction_description.startswith('tr'))

In [ ]:
print((TR.count(), len(TR.columns)))

(25, 14)


In [ ]:
TR = TR.withColumn("UTR", split(col("transaction_description"), "/").getItem(0))

In [ ]:
TR.show()

+----------------+--------------------+-----------------------+------------+-------------+---------------+-----------------+-------------------+-------+------+-------------+----------+----+--------------------+--------------------+
|transaction_date|          value_date|transaction_description|debit_amount|credit_amount|running_balance|pg_split_verified|wire_split_verified|product|reason|verified_flag|split_part|_c12|                _c13|                 UTR|
+----------------+--------------------+-----------------------+------------+-------------+---------------+-----------------+-------------------+-------+------+-------------+----------+----+--------------------+--------------------+
|         31:00.0|2022-01-21 00:00:...|   tr6e8cb25ded27491...|    2.41E+24|    339485.96|           null|      720607600.7|         PG Confirm|   null|  null|           PG|      null|   2|tr6e8cb25ded27491...|tr6e8cb25ded27491...|
|         21:00.0|2022-01-21 00:00:...|   trf191d96f3220447...|    2.41E

22/03/29 17:36:54 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: transaction_date, value_date, transaction_description, debit_amount, credit_amount, running_balance, pg_split_verified, wire_split_verified, product, reason, verified_flag, split_part, , 
 Schema: transaction_date, value_date, transaction_description, debit_amount, credit_amount, running_balance, pg_split_verified, wire_split_verified, product, reason, verified_flag, split_part, _c12, _c13
Expected: _c12 but found: 
CSV file: file:///media/easebuzz/DATA/Code/raw_files/Jan%20bank_stmt_final.csv


In [ ]:
twentytwo = bstatements.filter(bstatements.transaction_description.startswith('2022'))

In [ ]:
print((twentytwo.count(), len(twentytwo.columns)))

(25, 14)


In [ ]:
twentytwo = twentytwo.withColumn("UTR", split(col("transaction_description"), "/").getItem(0))

In [ ]:
twentytwo.show()

+----------------+--------------------+-----------------------+------------+-------------+---------------+-----------------+-------------------+-------+------+-------------+----------+----+-----------+--------------------+
|transaction_date|          value_date|transaction_description|debit_amount|credit_amount|running_balance|pg_split_verified|wire_split_verified|product|reason|verified_flag|split_part|_c12|       _c13|                 UTR|
+----------------+--------------------+-----------------------+------------+-------------+---------------+-----------------+-------------------+-------+------+-------------+----------+----+-----------+--------------------+
|         34:00.0|2022-01-21 00:00:...|   20220121281709364...|    2.41E+24|      1357600|           null|      713530325.1|               null|   null|  null|         null|      null|null|2.02201E+31|20220121281709364...|
|         00:00.0|2022-01-21 00:00:...|   20220121685278152...|    2.41E+24|        16953|           null|  

22/03/29 18:04:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: transaction_date, value_date, transaction_description, debit_amount, credit_amount, running_balance, pg_split_verified, wire_split_verified, product, reason, verified_flag, split_part, , 
 Schema: transaction_date, value_date, transaction_description, debit_amount, credit_amount, running_balance, pg_split_verified, wire_split_verified, product, reason, verified_flag, split_part, _c12, _c13
Expected: _c12 but found: 
CSV file: file:///media/easebuzz/DATA/Code/raw_files/Jan%20bank_stmt_final.csv


In [ ]:
pre_test= bstatements.filter(bstatements.transaction_description.startswith('PRE'))

In [ ]:
print((pre_test.count(), len(pre_test.columns)))

(11, 14)


In [ ]:
pre_test = pre_test.withColumn("UTR", split(col("transaction_description"), " ").getItem(2))

In [ ]:
pre_test.show()

+----------------+--------------------+-----------------------+------------+-------------+---------------+-----------------+-------------------+-------+------+-------------+----------+----+----+-------------+
|transaction_date|          value_date|transaction_description|debit_amount|credit_amount|running_balance|pg_split_verified|wire_split_verified|product|reason|verified_flag|split_part|_c12|_c13|          UTR|
+----------------+--------------------+-----------------------+------------+-------------+---------------+-----------------+-------------------+-------+------+-------------+----------+----+----+-------------+
|         29:00.0|2022-01-19 00:00:...|   PRE ARB 135872813361|    1.36E+11|          100|           null|      745679466.6|               null|   null|  null|           PG|      null|   2|null| 135872813361|
|         29:00.0|2022-01-19 00:00:...|   PRE ARB 135887091009|    1.36E+11|        15000|           null|      745656946.6|               null|   null|  null|     

22/03/29 18:23:44 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: transaction_date, value_date, transaction_description, debit_amount, credit_amount, running_balance, pg_split_verified, wire_split_verified, product, reason, verified_flag, split_part, , 
 Schema: transaction_date, value_date, transaction_description, debit_amount, credit_amount, running_balance, pg_split_verified, wire_split_verified, product, reason, verified_flag, split_part, _c12, _c13
Expected: _c12 but found: 
CSV file: file:///media/easebuzz/DATA/Code/raw_files/Jan%20bank_stmt_final.csv


# --------------------------------------------------------------------------------------------------------------------------------

In [37]:
receipt_new = spark.read.csv("/media/easebuzz/DATA/Code/raw_files/YESUPI_RECEIPT-Jan-2022.csv",header=True)

In [38]:
receipt_new.show()

+-------------------+--------------------+------------+--------+--------------------+-------+
|   Transaction Date|         Description|      Amount|C.D.Flag|             Ref .No|   Wire|
+-------------------+--------------------+------------+--------+--------------------+-------+
|2022-01-01 11:55:00|UPI EASEBUZZ PVT ...| 118180510.1|       C|                null|1214392|
|2022-01-01 11:31:00|RTGS Cr-ICIC00001...|  3747053.52|       C|ICICR220220101000...|   null|
|2022-01-01 11:31:00|RTGS Cr-ICIC00001...|  2229021.41|       C|ICICR220220101000...|   null|
|2022-01-02 00:00:00|UPI Transaction d...|119158649.85|       C|                null|1442203|
|2022-01-02 00:00:00|          YES CARD-1|  3263475.11|       C|                null|   null|
|2022-01-02 00:00:00|          YES CARD-2| 12085683.24|       C|                null|   null|
|2022-01-03 00:00:00|          YES CARD-1|  3547723.42|       C|                null|   null|
|2022-01-03 00:00:00|          YES CARD-2| 12901925.55|     

In [39]:
tempList = [] 
for col in receipt_new.columns:
    new_name = col.strip()
    new_name = "".join(new_name.split())
    new_name = new_name.replace('.','') # EDIT
    tempList.append(new_name) #Edit02
print(tempList)
receipt_new = receipt_new.toDF(*tempList)

['TransactionDate', 'Description', 'Amount', 'CDFlag', 'RefNo', 'Wire']


In [41]:
receipt_new= receipt_new.select('RefNo')

In [42]:
receipt_new.printSchema()

root
 |-- RefNo: string (nullable = true)



In [43]:
bankstatement1= spark.read.options(header = True,delimiter='|') \
.csv("/media/easebuzz/DATA/Code/raw_files/YESNSTAT_01Jan 22_to_10Jan22 .csv")

bankstatement2= spark.read.options(header = True,delimiter='|') \
.csv("/media/easebuzz/DATA/Code/raw_files/YESNSTAT_11Jan_to_20Jan22.csv")

bankstatement3= spark.read.options(header = True,delimiter='|') \
.csv("/media/easebuzz/DATA/Code/raw_files/YESNSTAT_21Jan_to_31Jan22.csv")

In [44]:
disDF = bankstatement1.union(bankstatement2).distinct()
disDF = disDF.union(bankstatement3).distinct()
#disDF = disDF.union(mark4).distinct()
print(disDF.count())

2434468


In [45]:
disDF.printSchema()

root
 |-- Transaction Date: string (nullable = true)
 |-- Value Date: string (nullable = true)
 |-- Transaction Description: string (nullable = true)
 |-- Referance no: string (nullable = true)
 |-- Debit Amount: string (nullable = true)
 |-- Credit Amount: string (nullable = true)
 |-- Closing Balance: string (nullable = true)
 |-- Remark: string (nullable = true)



In [46]:
tempList = [] 
for col in disDF.columns:
    new_name = col.strip()
    new_name = "".join(new_name.split())
    new_name = new_name.replace('.','') # EDIT
    tempList.append(new_name) #Edit02
print(tempList)
disDF = disDF.toDF(*tempList)

['TransactionDate', 'ValueDate', 'TransactionDescription', 'Referanceno', 'DebitAmount', 'CreditAmount', 'ClosingBalance', 'Remark']


In [47]:
upi = disDF.filter(disDF.TransactionDescription.startswith('UPI/'))
upi = upi.withColumn("UTR", split(sf.col("TransactionDescription"), "/").getItem(1))
upi_final_df = upi.select('TransactionDescription','UTR')

In [48]:
upi_final_df.printSchema()

root
 |-- TransactionDescription: string (nullable = true)
 |-- UTR: string (nullable = true)



In [50]:
print(upi_final_df.count())

178927


In [51]:
Matched=upi_final_df.join(receipt_new,upi_final_df.UTR == receipt_new.RefNo,"inner")


In [52]:
Matched.count()

5029

In [53]:
distinctDF = Matched.distinct()
print("Distinct count: "+str(distinctDF.count()))
distinctDF.show(truncate=False)

Distinct count: 5029


+-------------------------------------------------------------------------------------------------------------+------------+------------+
|TransactionDescription                                                                                       |UTR         |RefNo       |
+-------------------------------------------------------------------------------------------------------------+------------+------------+
|UPI/200381254248/RWU5SAUJNY/From:q2pay@yesbank/To:9711168923@ybl/Easebuzz                                    |200381254248|200381254248|
|UPI/201502695861/R15JANK17RUNH3VT/From:rummygoldrackbrainapp@yesbank/To:anuragverma7080508304@okaxis/Easebuzz|201502695861|201502695861|
|UPI/201807564159/R18JANKOKHE8RSFE/From:rummymastergameinn@yesbank/To:sarojachandfule5@okaxis/Easebuzz        |201807564159|201807564159|
|UPI/201502699641/R15JAN201465932697/From:rummymastergameinn@yesbank/To:7896587579@paytm/Easebuzz             |201502699641|201502699641|
|UPI/201705710958/R17JANK8EZG1NTSA

# ----------------------------------------------------------------------------------------------------------------------------

In [ ]:
receipt_new= receipt_new.select('Description')

In [ ]:
print(receipt_new.count())

5283


In [ ]:
matchinggg = receipt_new.filter(receipt_new.Description.startswith('UPI/'))
#print(matchinggg.count())
upi = matchinggg.withColumn("UTR", split(sf.col("Description"), "/").getItem(1))
upi_final_df = upi.select('Description','UTR')

In [ ]:
print(upi_final_df.count())

5046


# ----------------------------------------------------------------------------------------------------------------------------

In [ ]:
Matching = spark.read.csv("/media/easebuzz/DATA/Code/raw_files/Matching_result.csv",header = True)

In [ ]:
Matching.printSchema()

root
 |-- transactiondate: string (nullable = true)
 |-- transactiondescription: string (nullable = true)
 |-- remark: string (nullable = true)
 |-- debitamount: string (nullable = true)
 |-- creditamount: string (nullable = true)
 |-- utr: string (nullable = true)
 |-- refno: string (nullable = true)



In [ ]:
Matching= Matching.select('transactiondescription')

In [ ]:
print(Matching.count())

2176


In [ ]:
Unmatchinggg= receipt_new.subtract(Matching)

In [ ]:
receipt_new.printSchema()

root
 |-- Description: string (nullable = true)



In [ ]:
print(Unmatchinggg.count())

5177


In [ ]:
Unmatchinggg = Unmatchinggg.filter(Unmatchinggg.Description.startswith('UPI/'))
upi = Unmatchinggg.withColumn("UTR", split(sf.col("Description"), "/").getItem(1))
upi_final_df = upi.select('Description','UTR')

In [ ]:
spark.version

'3.2.1'

# ------------------------------------------------------------------------------------------------------------

In [11]:
Statements = disDF

In [12]:
Statements.show(10)

+----------------+----------+----------------------+------------+-----------+------------+---------------+------+
| TransactionDate| ValueDate|TransactionDescription| Referanceno|DebitAmount|CreditAmount| ClosingBalance|Remark|
+----------------+----------+----------------------+------------+-----------+------------+---------------+------+
|01-01-2022 00:06|01-01-2022|  IMPS/NA/XXXX2453/...|200100769002|      59.78|        null|60,95,00,340.01|  null|
|01-01-2022 00:09|01-01-2022|  IMPS/NA/XXXX5042/...|200100772905|     768.00|        null|60,92,50,932.64|  null|
|01-01-2022 00:09|01-01-2022|  IMPS/NA/XXXX9092/...|200100772920|     576.00|        null|60,92,48,851.76|  null|
|01-01-2022 00:13|01-01-2022|  IMPS/NA/XXXX3604/...|200100777425|     153.86|        null|60,89,78,190.48|  null|
|01-01-2022 00:19|01-01-2022|  IMPS/NA/XXXX9408/...|200100782546|    1001.56|        null|60,85,60,747.31|  null|
|01-01-2022 00:20|01-01-2022|  IMPS/NA/XXXX0003/...|200100783552|    1099.56|        nul

In [13]:
Statements.count()

2434468

In [121]:
tempList = [] 
for col in Statements.columns:
    new_name = col.strip()
    new_name = "".join(new_name.split())
    new_name = new_name.replace('.','') # EDIT
    tempList.append(new_name) #Edit02
print(tempList)
Statements = Statements.toDF(*tempList)

['TransactionDate', 'ValueDate', 'TransactionDescription', 'Referanceno', 'DebitAmount', 'CreditAmount', 'ClosingBalance', 'Remark']


In [17]:
wire_ledger = spark.read.csv("/media/easebuzz/DATA/Code/raw_files/ledger_utr_data_final_txn_date.csv",header = True)

In [18]:
wire_ledger.printSchema()

root
 |-- unique_transaction_reference: string (nullable = true)
 |-- transaction_amount: string (nullable = true)
 |-- transaction_date: string (nullable = true)



In [19]:
wire_ledger.count()

2326153

In [20]:
temp = Statements.filter(Statements.TransactionDescription.startswith('UPI/'))

upi = temp.withColumn("UTR", split(sf.col("TransactionDescription"), "/").getItem(1))
upi_final_df = upi.select('TransactionDate','TransactionDescription','Remark','DebitAmount','CreditAmount','UTR')

temp1 = Statements.filter(Statements.TransactionDescription.startswith('IMPS/'))
Imps = temp1.withColumn("UTR", split(sf.col("TransactionDescription"), "/").getItem(3))
Imps_final = Imps.withColumn("UTR", split(sf.col("UTR"), ":").getItem(1))
Imps_final_df = Imps_final.select('TransactionDate','TransactionDescription','Remark','DebitAmount','CreditAmount','UTR')

temp2 = Statements.filter(Statements.TransactionDescription.startswith('211'))
pattern_211 = temp2.withColumn("UTR", split(sf.col("TransactionDescription"), "/").getItem(1))
pattern_211_final_df = pattern_211.select('TransactionDate','TransactionDescription','Remark','DebitAmount','CreditAmount','UTR')

temp3 = Statements.filter(Statements.TransactionDescription.startswith('REV OF IMPS'))
rev_of_imps = temp3.withColumn("UTR", split(sf.col("TransactionDescription"), ":").getItem(2))
rev_of_imps_final_df = rev_of_imps.select('TransactionDate','TransactionDescription','Remark','DebitAmount','CreditAmount','UTR')

temp4 = Statements.filter(Statements.TransactionDescription.startswith('NEFT O/W'))
neft_o_w = temp4.withColumn("UTR", split(sf.col("TransactionDescription"), " ").getItem(2))
neft_o_w_final_df = neft_o_w.select('TransactionDate','TransactionDescription','Remark','DebitAmount','CreditAmount','UTR')

temp5 = Statements.filter(Statements.TransactionDescription.startswith('REV:UPI'))
rev_upi = temp5.withColumn("UTR", split(sf.col("TransactionDescription"), "/").getItem(1))
rev_upi_final_df = rev_upi.select('TransactionDate','TransactionDescription','Remark','DebitAmount','CreditAmount','UTR')

temp6 = Statements.filter(Statements.TransactionDescription.startswith('Funds Trf'))
funds_trf = temp6.withColumn("UTR", split(sf.col("TransactionDescription"), "/").getItem(1))
funds_trf_final = funds_trf.withColumn("UTR", split(sf.col("UTR"), ":").getItem(1))
funds_trf_final_df = funds_trf_final.select('TransactionDate','TransactionDescription','Remark','DebitAmount','CreditAmount','UTR')

temp7 = Statements.filter(Statements.TransactionDescription.startswith('IMPS/NA'))
store = temp7.filter(sf.col("TransactionDescription").contains("REV"))
store_imps_rev = store.select('TransactionDate','TransactionDescription','Remark','DebitAmount','CreditAmount')
Imps_reverse = store_imps_rev.withColumn("UTR", split(sf.col("TransactionDescription"), "/").getItem(4))
Imps_reverse_final = Imps_reverse.withColumn("UTR", split(sf.col("UTR"), ":").getItem(1))
Imps_reverse_final_df = Imps_reverse_final.select('TransactionDate','TransactionDescription','Remark','DebitAmount','CreditAmount','UTR')

temp8 = Statements.filter(Statements.TransactionDescription.startswith('NEFT-RETURN'))
Neft_return = temp8.withColumn("UTR", split(sf.col("TransactionDescription"), "-").getItem(2))
Neft_return_final_df = Neft_return.select('TransactionDate','TransactionDescription','Remark','DebitAmount','CreditAmount','UTR')

temp9 = Statements.filter(Statements.TransactionDescription.startswith('tr'))
tr = temp9.withColumn("UTR", split(sf.col("TransactionDescription"), "/").getItem(0))
tr_final_df = tr.select('TransactionDate','TransactionDescription','Remark','DebitAmount','CreditAmount','UTR')
#tr_final_df.show()

temp10 = Statements.filter(Statements.TransactionDescription.startswith('2022'))
twentytwo = temp10.withColumn("UTR", split(sf.col("TransactionDescription"), "/").getItem(0))
twentytwo_final_df = twentytwo.select('TransactionDate','TransactionDescription','Remark','DebitAmount','CreditAmount','UTR')

temp11= Statements.filter(Statements.TransactionDescription.startswith('PRE'))
pre_test = temp11.withColumn("UTR", split(sf.col("TransactionDescription"), " ").getItem(2))
pre_test_final_df = pre_test.select('TransactionDate','TransactionDescription','Remark','DebitAmount','CreditAmount','UTR')

temp12= Statements.filter(Statements.TransactionDescription.startswith('CBK/'))
cbk = temp12.withColumn("UTR", split(sf.col("TransactionDescription"), "/").getItem(2))
cbk_final_df = cbk.select('TransactionDate','TransactionDescription','Remark','DebitAmount','CreditAmount','UTR')

disDF = upi_final_df.union(Imps_final_df).distinct()
disDF = disDF.union(pattern_211_final_df).distinct()
disDF = disDF.union(rev_of_imps_final_df).distinct()
disDF = disDF.union(neft_o_w_final_df).distinct()
disDF = disDF.union(rev_upi_final_df).distinct()
disDF = disDF.union(funds_trf_final_df).distinct()
disDF = disDF.union(Imps_reverse_final_df).distinct()
disDF = disDF.union(Neft_return_final_df).distinct()
disDF = disDF.union(tr_final_df).distinct()
disDF = disDF.union(twentytwo_final_df).distinct()
disDF = disDF.union(pre_test_final_df).distinct()
final_df = disDF.union(cbk_final_df).distinct()


In [21]:
final_df.count()

2495637

In [22]:
Matched_records=final_df.join(wire_ledger,final_df.UTR == wire_ledger.unique_transaction_reference,"inner")

In [25]:
Matched_records.count()

2245206

In [26]:
Matched_records.printSchema()

root
 |-- TransactionDate: string (nullable = true)
 |-- TransactionDescription: string (nullable = true)
 |-- Remark: string (nullable = true)
 |-- DebitAmount: string (nullable = true)
 |-- CreditAmount: string (nullable = true)
 |-- UTR: string (nullable = true)
 |-- unique_transaction_reference: string (nullable = true)
 |-- transaction_amount: string (nullable = true)
 |-- transaction_date: string (nullable = true)



In [32]:
Matched_records.write.csv('/media/easebuzz/DATA/Code/matching result/Matched/wirematched.csv')

In [28]:
temp_mr=Matched_records.select('TransactionDescription')

In [29]:
temp_umr=Statements.select('TransactionDescription')

In [30]:
Unmatched_result=temp_umr.subtract(temp_mr)

In [36]:
Unmatched_result.write.csv('/media/easebuzz/DATA/Code/matching result/Un-Matched/wire_unmatched.csv')